# Kaggle & kaggle api 
* download datasets from Kaggle.com 
* upload submission.csv-files

In [6]:
import numpy as np
import pandas as pd
import os
import kaggle
import warnings
from pathlib import Path

warnings.simplefilter('ignore')

In [9]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: 
    path = Path('./Data/house-prices-advanced-regression-techniques')
else:
    path = Path('house-prices-advanced-regression-techniques')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 199k/199k [00:00<00:00, 2.14MB/s]

In [10]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: 
    path = Path('./Data/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 34.1k/34.1k [00:00<00:00, 2.52MB/s]

In [12]:
import json

kaggle_json_path = '../../API_Keys/kaggle.json'
with open(kaggle_json_path) as f:
    kaggle_json_content = json.load(f)

kaggle_api_key = kaggle_json_content['key']



FileNotFoundError: [Errno 2] No such file or directory: '../../API_Keys/kaggle.json'

In [13]:
X = pd.read_csv('./titanic/train.csv')
X.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


X = pd.read_csv('./titanic/train.csv')
display(X.head())
X = X.replace({'male':1, 'female':0})
y = X['Survived'].values
X.drop(columns = ['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
X = pd.get_dummies(X,dummy_na=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

params = {
    'max_depth': 6,
    'learning_rate': 0.01,
    'n_estimators': 300,
    'scale_pos_weight': 1,
}

model_xgb = XGBClassifier(**params, booster='gbtree',)
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict_proba(X_test)[:,1]
print(f"ROC (X_train): {metrics.roc_auc_score(y_train, model_xgb.predict_proba(X_train)[:,1])}")
print(f"ROC (X_test): {metrics.roc_auc_score(y_test, y_pred)}")

X_test = pd.read_csv('./titanic/test.csv')
X_test = X_test.replace({'male':1, 'female':0})
passengerids = X_test['PassengerId']
X_test.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
X_test = pd.get_dummies(X_test,dummy_na=True)
display(X_test.head())
y_pred = model_xgb.predict(X_test)
pd.DataFrame.from_dict({'PassengerId': passengerids, 'Survived': y_pred}).to_csv('./titanic/submission.csv', index=False)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


ROC (X_train): 0.9527357607247886
ROC (X_test): 0.8716883116883116


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3,1,34.5,0,0,7.8292,0,1,0,0
1,3,0,47.0,1,0,7.0000,0,0,1,0
2,2,1,62.0,0,0,9.6875,0,1,0,0
3,3,1,27.0,0,0,8.6625,0,0,1,0
4,3,0,22.0,1,1,12.2875,0,0,1,0


In [15]:
# https://www.kaggle.com/competitions/microsoft-malware-prediction/discussion/76245#448176 

import kaggle
import pandas as pd

# Make a Kaggle submission
kaggle.api.competition_submit(competition='titanic',
                              file_name='./titanic/submission.csv',
                              message=f"xbgoost, dummies for 'Embarked' and {str(params)}")

100%|██████████| 2.77k/2.77k [00:00<00:00, 6.32kB/s]


Successfully submitted to Titanic - Machine Learning from Disaster